How to Use This Version for Checkpoint Resume in Kaggle

Before running this notebook for the second time (to resume training from a checkpoint), follow the steps below carefully.

1. Download the latest checkpoint file
Download the file named checkpoint_latest.pth from the previous version of your notebook or experiment.

2. Upload the checkpoint to Kaggle Input Directory
Place the downloaded file inside your Kaggle input path, for example:
/kaggle/input/path1/pytorch/default/1/checkpoint_latest.pth

3. Run the following code cell before starting training
This code will copy the checkpoint file to the working directory (/kaggle/working/checkpoints) so that training can resume from the saved state.

4. Resume Training
After the checkpoint file is copied successfully, running the rest of the notebook will automatically start training from the previous checkpoint instead of starting from scratch.



## you can change the dataset and the attack type by simply changing the name in the args.py file.no need to modify anything else.

In [ ]:
import os
import shutil

# Source and destination paths
src = "/kaggle/input/path1/pytorch/default/1/checkpoint_latest.pth"
dst_dir = "/kaggle/working/checkpoints"
dst = os.path.join(dst_dir, "checkpoint_latest.pth")

# Step 1: Check if source file exists
if not os.path.exists(src):
    print(f"❌ Source file not found: {src}")
else:
    print(f"✅ Found source file: {src}")

    # Step 2: Ensure destination directory exists
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
        print(f"📂 Created destination directory: {dst_dir}")
    else:
        print(f"📁 Destination directory already exists: {dst_dir}")

    # Step 3: Copy the file
    shutil.copy(src, dst)
    print(f"✅ Copied file to: {dst}")

    # Step 4: List all files in destination
    files = os.listdir(dst_dir)
    if files:
        print("\n📄 Files in /kaggle/working/checkpoints:")
        for f in files:
            print(" ├──", f)
    else:
        print("⚠ Destination directory is empty (unexpected).")


In [1]:
import os

# Target directory
base_dir = "/kaggle/working/FedBn-PyTorch"
os.makedirs(base_dir, exist_ok=True)

# Python files to create
files = ["main.py", "server.py", "client.py", "model.py", "get_data.py", "args.py"]

# Create each file if not exists
for file in files:
    file_path = os.path.join(base_dir, file)
    if not os.path.exists(file_path):
        with open(file_path, "w") as f:
            f.write(f"# {file} — auto-created placeholder\n")
        print(f" Created: {file_path}")
    else:
        print(f" Already exists: {file_path}")

print("\n Folder and files ready in:", base_dir)


 Created: /kaggle/working/FedBn-PyTorch/main.py
 Created: /kaggle/working/FedBn-PyTorch/server.py
 Created: /kaggle/working/FedBn-PyTorch/client.py
 Created: /kaggle/working/FedBn-PyTorch/model.py
 Created: /kaggle/working/FedBn-PyTorch/get_data.py
 Created: /kaggle/working/FedBn-PyTorch/args.py

 Folder and files ready in: /kaggle/working/FedBn-PyTorch


In [2]:
file_path = "/kaggle/working/FedBn-PyTorch/args.py"

new_code = '''
# ========================================
# args.py — FedBN Configuration 
# ========================================
import argparse
import torch

DATASET_CONFIGS = {
    'pathmnist': {'num_classes': 9, 'input_channels': 3},
    'tissuemnist': {'num_classes': 8, 'input_channels': 1},
    'organamnist': {'num_classes': 11, 'input_channels': 1},
    'octmnist': {'num_classes': 4, 'input_channels': 1},
}

def args_parser():
    parser = argparse.ArgumentParser(description="FedBN Config (4 Dataset Compatible)")
    
    # -------------------------------
    # Dataset
    # -------------------------------
    parser.add_argument('--dataset', type=str, default='organamnist',
                        choices=['pathmnist', 'tissuemnist', 'organamnist', 'octmnist'],
                        help='MedMNIST dataset to use')
    parser.add_argument('--use_combined', action='store_true', 
                        help='Use train+val+test combined')

    # -------------------------------
    # Federated Learning Parameters
    # -------------------------------
    parser.add_argument('--E', type=int, default=5, help='local epochs')
    parser.add_argument('--r', type=int, default=50, help='communication rounds')
    parser.add_argument('--K', type=int, default=5, help='number of clients')
    parser.add_argument('--C', type=float, default=1.0, help='client sampling rate')
    parser.add_argument('--B', type=int, default=32, help='batch size')

    # -------------------------------
    # Model Parameters
    # -------------------------------
    parser.add_argument('--clip_model', type=str, default='ViT-B/32', help='CLIP model variant')
    parser.add_argument('--freeze_clip', action='store_true', help='Freeze CLIP backbone')
    parser.add_argument('--dropout', type=float, default=0.5, help='dropout rate')

    # -------------------------------
    # Optimizer Settings
    # -------------------------------
    parser.add_argument('--lr', type=float, default=0.003, help='learning rate')
    parser.add_argument('--optimizer', type=str, default='sgd', help='optimizer type')
    parser.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum for SGD')

    # -------------------------------
    # FedBN Specific
    # -------------------------------
    parser.add_argument('--exclude_bn_from_agg', action='store_true',
                        help='Exclude BN params from aggregation')
    parser.add_argument('--freeze_bn', action='store_true', help='Freeze BN layers')
    
    # -------------------------------
    # Others
    # -------------------------------
    parser.add_argument('--checkpoint_dir', type=str, default='./checkpoints_fedbn')
    parser.add_argument('--device', default=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    parser.add_argument('--dominant_ratio', type=float, default=0.7, help='dominant ratio for Non-IID split')

    args = parser.parse_args(args=[])

    if args.dataset in DATASET_CONFIGS:
        cfg = DATASET_CONFIGS[args.dataset]
        args.num_classes = cfg['num_classes']
        args.input_channels = cfg['input_channels']
    else:
        raise ValueError(f"Unsupported dataset {args.dataset}")

    return args

'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" args.py written successfully!")
print(f" File path: {file_path}")


 args.py written successfully!
 File path: /kaggle/working/FedBn-PyTorch/args.py


In [3]:
file_path = "/kaggle/working/FedBn-PyTorch/get_data.py"

new_code = r'''
# ========================================
# get_data.py — Balanced Non-IID Loader (4 Dataset Compatible)
# ========================================
import os
import numpy as np
import torch
import pickle
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision import transforms
from medmnist.dataset import PathMNIST, TissueMNIST, OrganAMNIST, OCTMNIST

DATASET_MAP = {
    'pathmnist': PathMNIST,
    'tissuemnist': TissueMNIST,
    'organamnist': OrganAMNIST,
    'octmnist': OCTMNIST,
}

def get_transforms():
    """Unified transforms (grayscale → RGB for CLIP)"""
    train_t = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    test_t = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    return train_t, test_t

def balanced_noniid_split(dataset, num_clients, dominant_ratio=0.7):
    labels = np.array([dataset[i][1].item() for i in range(len(dataset))])
    n_classes = len(np.unique(labels))
    client_indices = [[] for _ in range(num_clients)]

    class_indices = {c: np.where(labels == c)[0].tolist() for c in range(n_classes)}
    for c in class_indices: np.random.shuffle(class_indices[c])

    for client_id in range(num_clients):
        dom_class = client_id % n_classes
        n_dom = int(len(class_indices[dom_class]) * dominant_ratio)
        client_indices[client_id].extend(class_indices[dom_class][:n_dom])
        class_indices[dom_class] = class_indices[dom_class][n_dom:]

    for c in range(n_classes):
        rem = class_indices[c]
        np.random.shuffle(rem)
        others = [i for i in range(num_clients) if i % n_classes != c]
        for i, idx in enumerate(rem):
            client_indices[others[i % len(others)]].append(idx)
    for i in range(num_clients):
        np.random.shuffle(client_indices[i])
    return client_indices

def load_medmnist_data(args):
    train_t, test_t = get_transforms()
    root = './data/medmnist'
    os.makedirs(root, exist_ok=True)
    Dataset = DATASET_MAP[args.dataset]

    train = Dataset(root=root, split='train', download=True, transform=train_t)
    val = Dataset(root=root, split='val', download=True, transform=train_t)
    test = Dataset(root=root, split='test', download=True, transform=test_t)
    combined = ConcatDataset([train, val])

    cache = f'./data/medmnist/client_idx_{args.dataset}_K{args.K}_dr{args.dominant_ratio}_fedbn.pkl'
    if os.path.exists(cache):
        with open(cache, 'rb') as f:
            client_idx = pickle.load(f)
    else:
        client_idx = balanced_noniid_split(combined, args.K, args.dominant_ratio)
        with open(cache, 'wb') as f:
            pickle.dump(client_idx, f)

    client_loaders = [
        DataLoader(Subset(combined, idx), batch_size=args.B, shuffle=True, num_workers=0)
        for idx in client_idx
    ]
    test_loader = DataLoader(test, batch_size=args.B, shuffle=False, num_workers=0)
    return client_loaders, test_loader

'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" get_data.py created for FedBN!")


 get_data.py created for FedBN!


In [4]:
file_path = "/kaggle/working/FedBn-PyTorch/client.py"

new_code = """# ========================================
# client.py — Client-side helper for FedBN (with gradient clipping)
# ========================================
import torch
from torch import nn
import torch.optim as optim
from tqdm import tqdm
import copy

class Client:
    def __init__(self, model, train_loader, device, val_loader=None,
                 lr=0.001, weight_decay=1e-4, use_fedbn=True):
        \"\"\"
        Args:
            use_fedbn (bool): If True, BN layers remain local (not aggregated)
        \"\"\"
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.lr = lr
        self.weight_decay = weight_decay
        self.use_fedbn = use_fedbn

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def compute_accuracy(self, loader):
        \"\"\"Compute accuracy on given DataLoader\"\"\"
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(self.device), labels.to(self.device)
                labels = labels.squeeze()
                if labels.dim() == 0:
                    labels = labels.unsqueeze(0)
                if labels.size(0) == 0:
                    continue
                outputs = self.model(images)
                if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total if total > 0 else 0

    def train(self, epochs=1):
        \"\"\"Local training (BN layers remain local; only non-BN parameters aggregated)\"\"\"
        print()
        print(" Training client model with FedBN objective...")
        
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0.0
            batch_count = 0

            pbar = tqdm(self.train_loader,
                        desc=f"  Epoch {epoch+1}/{epochs}",
                        ncols=100,
                        leave=False)

            for images, labels in pbar:
                images, labels = images.to(self.device), labels.to(self.device)
                labels = labels.squeeze()
                if labels.dim() == 0:
                    labels = labels.unsqueeze(0)
                if labels.size(0) == 0:
                    continue

                self.optimizer.zero_grad()
                outputs = self.model(images)
                if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                loss = self.criterion(outputs, labels)
                
                # Check for NaN and skip if found
                if torch.isnan(loss):
                    print("  NaN detected, skipping batch")
                    continue
                
                loss.backward()
                
                # Gradient clipping for stability
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.optimizer.step()

                total_loss += loss.item()
                batch_count += 1
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            avg_loss = total_loss / batch_count if batch_count > 0 else 0
            train_acc = self.compute_accuracy(self.train_loader)
            log_msg = f"  Epoch {epoch+1:2d}/{epochs} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}%"

            if self.val_loader is not None:
                val_acc = self.compute_accuracy(self.val_loader)
                val_loss, val_batches = 0.0, 0
                self.model.eval()
                with torch.no_grad():
                    for images, labels in self.val_loader:
                        images, labels = images.to(self.device), labels.to(self.device)
                        labels = labels.squeeze()
                        if labels.dim() == 0:
                            labels = labels.unsqueeze(0)
                        if labels.size(0) == 0:
                            continue
                        outputs = self.model(images)
                        if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                            outputs = outputs.logits
                        loss = self.criterion(outputs, labels)
                        val_loss += loss.item()
                        val_batches += 1
                val_loss = val_loss / val_batches if val_batches > 0 else 0
                log_msg += f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%"

            print(log_msg)
        
        print(" Client local training complete (FedBN)")
        print()
        return self.model.state_dict()
"""

with open(file_path, "w") as f:
    f.write(new_code)

print(" client.py updated!")

 client.py updated!


In [5]:
file_path = "/kaggle/working/FedBn-PyTorch/model.py"

new_code = '''# ========================================
# model.py — CLIP-based FedBN (BN Local Only)
# ========================================
import torch
from torch import nn
import clip

class CLIPFedBNClassifier(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.clip_model, _ = clip.load(args.clip_model, device=args.device)

        # Freeze visual encoder
        for p in self.clip_model.parameters():
            p.requires_grad = False

        feature_dim = self.clip_model.visual.output_dim
        self.fc1 = nn.Linear(feature_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, args.num_classes)
        self.bn2 = nn.BatchNorm1d(args.num_classes)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(args.dropout)

    def forward(self, x):
        with torch.no_grad():
            img_feat = self.clip_model.encode_image(x).float()
        x = self.fc1(img_feat)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.bn2(x)
        return x

    def get_bn_params(self):
        """Return BN params (excluded from aggregation)"""
        bn_params = []
        for n, m in self.named_modules():
            if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
                for p in ['weight', 'bias', 'running_mean', 'running_var']:
                    bn_params.append(f"{n}.{p}")
        return bn_params

'''

with open(file_path, "w", newline='\n') as f:
    f.write(new_code)

print("model.py updated!")


model.py updated!


In [6]:
file_path = "/kaggle/working/FedBn-PyTorch/server.py"

new_code = '''# ========================================
# server.py - FedBN Server (CLIP-based, Auto Resume)
# ========================================
import copy
import random
import numpy as np
import torch
import os
import json
from model import CLIPFedBNClassifier as ImageClassifier
from get_data import load_medmnist_data
from client import Client
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error

class FedBNServer:
    def __init__(self, args):
        self.args = args
        os.makedirs(args.checkpoint_dir, exist_ok=True)
        self.current_round = 0
        self.best_global_acc = 0
        self.history = {'rounds': [], 'avg_accuracy': [], 'best_accuracy': []}

        #  Initialize global CLIP model (no 'name' param)
        self.global_model = ImageClassifier(args).to(args.device)

        # Collect BN parameter names for exclusion
        self.bn_params = self._get_bn_param_names()
        print(f\" Excluded BN parameters: {len(self.bn_params)} params\")

        # Create client models
        self.client_models = []
        for i in range(self.args.K):
            model = copy.deepcopy(self.global_model)
            self.client_models.append(model)

        # Load MedMNIST client datasets
        self.client_loaders, self.test_loader = load_medmnist_data(args)

        #  Auto resume if checkpoint exists
        latest_ckpt = os.path.join(args.checkpoint_dir, 'checkpoint_latest.pth')
        if os.path.exists(latest_ckpt):
            print(f\" Found checkpoint at {latest_ckpt}. Resuming training...\")
            self.load_checkpoint(latest_ckpt)
        else:
            print(\" No checkpoint found. Starting training from scratch.\")

    def _get_bn_param_names(self):
        bn_params = set()
        for name, module in self.global_model.named_modules():
            if isinstance(module, (torch.nn.BatchNorm2d, torch.nn.BatchNorm1d)):
                bn_params.update({
                    f\"{name}.weight\", f\"{name}.bias\",
                    f\"{name}.running_mean\", f\"{name}.running_var\",
                    f\"{name}.num_batches_tracked\"
                })
        return bn_params

    def _is_bn_param(self, param_name):
        return param_name in self.bn_params

    def dispatch(self, selected_clients):
        global_dict = self.global_model.state_dict()
        for idx in selected_clients:
            client_dict = self.client_models[idx].state_dict()
            for key in global_dict.keys():
                if not self._is_bn_param(key):
                    client_dict[key] = global_dict[key].clone()
            self.client_models[idx].load_state_dict(client_dict)

    def aggregate(self, selected_clients):
        total_samples = sum([len(self.client_loaders[idx].dataset) for idx in selected_clients])
        global_dict = self.global_model.state_dict()
        aggregated = {k: torch.zeros_like(v) for k, v in global_dict.items()}

        for idx in selected_clients:
            weight = len(self.client_loaders[idx].dataset) / total_samples
            client_dict = self.client_models[idx].state_dict()
            for key in aggregated.keys():
                if not self._is_bn_param(key):
                    aggregated[key] += client_dict[key] * weight
                else:
                    aggregated[key] = global_dict[key]

        self.global_model.load_state_dict(aggregated)

    def client_update(self, idx):
        client_model = self.client_models[idx]
        client_loader = self.client_loaders[idx]
        client_obj = Client(
            model=client_model,
            train_loader=client_loader,
            device=self.args.device,
            lr=self.args.lr,
            weight_decay=self.args.weight_decay,
            use_fedbn=True
        )
        client_obj.train(epochs=self.args.E)

    def test_global_model(self):
        self.global_model.eval()
        all_labels, all_preds = [], []

        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(self.args.device), labels.to(self.args.device)
                labels = labels.squeeze()
                outputs = self.global_model(images)
                if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                _, predicted = torch.max(outputs, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())

        all_labels, all_preds = np.array(all_labels), np.array(all_preds)
        acc = 100 * np.mean(all_labels == all_preds)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        rmse = np.sqrt(mean_squared_error(all_labels, all_preds))

        print(f\" Global Test — Acc: {acc:.2f}% | Prec: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f} | RMSE: {rmse:.3f}\")
        return acc

    def load_checkpoint(self, checkpoint_path):
        print(f\"\
 Loading checkpoint: {checkpoint_path}\")
        checkpoint = torch.load(checkpoint_path, map_location=self.args.device, weights_only=False)
        self.global_model.load_state_dict(checkpoint['server_state_dict'])
        if 'client_state_dicts' in checkpoint:
            for i, state_dict in enumerate(checkpoint['client_state_dicts']):
                self.client_models[i].load_state_dict(state_dict)
        self.current_round = checkpoint.get('round', 0)
        self.best_global_acc = checkpoint.get('best_global_acc', 0)
        self.history = checkpoint.get('history', {'rounds': [], 'avg_accuracy': [], 'best_accuracy': []})
        print(f\" Resumed from Round {self.current_round} | Best Acc: {self.best_global_acc:.2f}%\")

    def save_checkpoint(self, round_num, avg_acc):
        checkpoint = {
            'round': round_num,
            'server_state_dict': self.global_model.state_dict(),
            'client_state_dicts': [m.state_dict() for m in self.client_models],
            'best_global_acc': self.best_global_acc,
            'avg_accuracy': avg_acc,
            'history': self.history,
            'args': vars(self.args)
        }
       
        torch.save(checkpoint, os.path.join(self.args.checkpoint_dir, 'checkpoint_latest.pth'))
      
        with open(os.path.join(self.args.checkpoint_dir, 'training_history.json'), 'a') as f:
            json.dump(self.history, f, indent=4)

    def run(self):
        start_round = self.current_round
        for r in range(start_round, self.args.r):
            print(f'\
{\"=\"*60}')
            print(f'Round {r+1}/{self.args.r}')
            print(f'{\"=\"*60}')
            m = max(int(self.args.C * self.args.K), 1)
            selected_clients = random.sample(range(self.args.K), m)
            print(f\"Selected clients: {selected_clients}\")
            self.dispatch(selected_clients)
            for idx in selected_clients:
                self.client_update(idx)
            self.aggregate(selected_clients)
            avg_acc = self.test_global_model()
            self.history['rounds'].append(r+1)
            self.history['avg_accuracy'].append(avg_acc)
            self.history['best_accuracy'].append(max(self.best_global_acc, avg_acc))
            if avg_acc > self.best_global_acc:
                self.best_global_acc = avg_acc
                print(f\" NEW BEST: {self.best_global_acc:.2f}%\")
            self.current_round = r + 1
            self.save_checkpoint(r+1, avg_acc)
    
        print(f'\
{\"=\"*60}')
        print(f' Training Complete! Best Accuracy: {self.best_global_acc:.2f}%')
        print(f'{\"=\"*60}')
        return self.global_model


'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" FedBN server.py uppdated!")


 FedBN server.py uppdated!


In [7]:
file_path = "/kaggle/working/FedBn-PyTorch/main.py"

new_code = '''# ========================================
# main.py — Run FedBN with CLIP model (Auto Resume)
# ========================================
from args import args_parser
from server import FedBNServer
import torch
import os
import json
import matplotlib.pyplot as plt

def plot_training_history(history, save_path):
    """Plot training accuracy curves"""
    if not history['rounds']:
        print("  No training history to plot")
        return
    
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Average Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['rounds'], history['avg_accuracy'], 'b-', label='Avg Accuracy', linewidth=2)
    plt.plot(history['rounds'], history['best_accuracy'], 'r--', label='Best Accuracy', linewidth=2)
    plt.xlabel('Communication Round')
    plt.ylabel('Accuracy (%)')
    plt.title('Federated Learning Accuracy Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Accuracy Improvement
    plt.subplot(1, 2, 2)
    if len(history['avg_accuracy']) > 1:
        improvements = [history['avg_accuracy'][i] - history['avg_accuracy'][i-1] 
                        for i in range(1, len(history['avg_accuracy']))]
        plt.bar(history['rounds'][1:], improvements, alpha=0.7)
        plt.axhline(y=0, color='r', linestyle='-', linewidth=0.5)
        plt.xlabel('Communication Round')
        plt.ylabel('Accuracy Change (%)')
        plt.title('Round-to-Round Accuracy Change')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f' Training plot saved at: {save_path}')

def main():
    # Load arguments
    args = args_parser()
    
    # Print configuration
    print('\\n' + '='*60)
    print(' FEDBN TRAINING CONFIGURATION (CLIP Model)')
    print('='*60)
    print(f'Device: {args.device}')
    print(f'Clients: {args.K} | Rounds: {args.r} | Local Epochs: {args.E}')
    print(f'Batch Size: {args.B} | Learning Rate: {args.lr}')
    print(f'Non-IID: {args.dominant_ratio*100:.0f}% dominant class per client')
    print(f'Checkpoint Directory: {args.checkpoint_dir}')
    print('='*60)
    
    # Check if checkpoint exists (just for info - server will auto-resume)
    latest_checkpoint = os.path.join(args.checkpoint_dir, 'checkpoint_latest.pth')
    if os.path.exists(latest_checkpoint):
        print('\\n' + '='*60)
        print(' CHECKPOINT FOUND - Will auto-resume')
        print('='*60)
        try:
            checkpoint = torch.load(latest_checkpoint, map_location=args.device, weights_only=False)
            completed_rounds = checkpoint.get('round', 0)
            best_acc = checkpoint.get('best_global_acc', 0)
            print(f' Checkpoint Details:')
            print(f'   - Completed Rounds: {completed_rounds}/{args.r}')
            print(f'   - Best Accuracy: {best_acc:.2f}%')
        except Exception as e:
            print(f'  Error reading checkpoint: {e}')
    else:
        print('\\n' + '='*60)
        print(' No checkpoint found. Starting fresh training...')
        print('='*60)
    
    # Initialize FedBN server (auto-resumes if checkpoint exists)
    server = FedBNServer(args)
    
    # Run federated training
    final_model = server.run()
    
    # Save final global model
    final_model_path = os.path.join(args.checkpoint_dir, 'final_global_model.pth')
    torch.save(final_model.state_dict(), final_model_path)
    print(f'\\n Final global model saved at: {final_model_path}')
    

    
    # Plot training curves
    plot_path = os.path.join(args.checkpoint_dir, 'training_plot.png')
    plot_training_history(server.history, plot_path)
    
    # Print summary
    print('\\n' + '='*60)
    print(' TRAINING SUMMARY')
    print('='*60)
    print(f' Best Global Accuracy: {server.best_global_acc:.2f}%')
    if server.history['avg_accuracy']:
        print(f' Final Round Accuracy: {server.history["avg_accuracy"][-1]:.2f}%')
        if len(server.history['avg_accuracy']) > 1:
            improvement = server.history['avg_accuracy'][-1] - server.history['avg_accuracy'][0]
            print(f' Total Improvement: {improvement:.2f}%')
    print(f' All checkpoints saved in: {args.checkpoint_dir}')
    print('='*60)

if __name__ == '__main__':
    main()
'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" FedBN main.py saved successfully!")

 FedBN main.py saved successfully!


In [8]:
!pip install medmnist --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency r

In [9]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-dk2ydheh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-dk2ydheh
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=56dfd9f67d4483512338965fefd20a53e154adefc036e112e19b505b08f44b5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-annf6ndu/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
!python /kaggle/working/FedBn-PyTorch/main.py


 FEDBN TRAINING CONFIGURATION (CLIP Model)
Device: cuda
Clients: 5 | Rounds: 50 | Local Epochs: 5
Batch Size: 32 | Learning Rate: 0.003
Non-IID: 70% dominant class per client
Checkpoint Directory: ./checkpoints_fedbn

 No checkpoint found. Starting fresh training...
100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 185MiB/s]
 Excluded BN parameters: 10 params
100%|███████████████████████████████████████| 38.2M/38.2M [00:52<00:00, 722kB/s]
 No checkpoint found. Starting training from scratch.
Round 1/50
Selected clients: [4, 0, 2, 1, 3]

 Training client model with FedBN objective...
  Epoch  1/5 | Loss: 1.0989 | Train Acc: 82.45%                                                     
  Epoch  2/5 | Loss: 0.7396 | Train Acc: 83.74%                                                     
  Epoch  3/5 | Loss: 0.6574 | Train Acc: 84.14%                                                     
  Epoch  4/5 | Loss: 0.6405 | Train Acc: 85.40%                                      